In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


,Adm_ID,Note_ID,Input_ID,Label,chartdate,charttime
0,22595853,10000032-RR-15,101 7749 1024 11290 2030 26033 28522 20791 214...,0,2180-05-06 23:00:00,2180-05-06 23:00:00
1,22595853,10000032-RR-15,101 2854 29454 3370 1012 1996 17324 2094 5761 ...,0,2180-05-06 23:00:00,2180-05-06 23:00:00
2,22595853,10000032-RR-15,101 1024 5107 3550 8810 1997 20118 13320 1998 ...,0,2180-05-06 23:00:00,2180-05-06 23:00:00
3,22595853,10000032-RR-16,101 12407 1024 1035 16731 2615 25022 12171 252...,0,2180-05-07 09:55:00,2180-05-07 09:55:00
4,22595853,10000032-RR-16,101 4663 1012 1037 17463 3217 11788 5397 2051 ...,0,2180-05-07 09:55:00,2180-05-07 09:55:00


In [ ]:
import pandas as pd
m4_train = pd.read_csv("/content/drive/My Drive/m4_train.csv")
m4_val = pd.read_csv("/content/drive/My Drive/m4_val.csv")
m4_test = pd.read_csv("/content/drive/My Drive/m4_test.csv")


In [ ]:
type(m4_train['Input_ID'])

pandas.core.series.Series

In [ ]:
import ast

# 假设m4_train是你的DataFrame
m4_train['Input_ID'] = m4_train['Input_ID'].apply(ast.literal_eval)
m4_val['Input_ID'] = m4_val['Input_ID'].apply(ast.literal_eval)
m4_test['Input_ID'] = m4_test['Input_ID'].apply(ast.literal_eval)

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
input_encoder = LabelEncoder()
m4_train['input_id_encoded'] = input_encoder.fit_transform(m4_train['Input_ID'])

adm_encoder = LabelEncoder()
m4_train['adm_id_encoded'] = adm_encoder.fit_transform(m4_train['Adm_ID'])

input_encoder = LabelEncoder()
m4_val['input_id_encoded'] = input_encoder.fit_transform(m4_val['Input_ID'])

adm_encoder = LabelEncoder()
m4_val['adm_id_encoded'] = adm_encoder.fit_transform(m4_val['Adm_ID'])

input_encoder = LabelEncoder()
m4_test['input_id_encoded'] = input_encoder.fit_transform(m4_test['Input_ID'])

adm_encoder = LabelEncoder()
m4_test['adm_id_encoded'] = adm_encoder.fit_transform(m4_test['Adm_ID'])

In [ ]:
type(m4_train['Adm_ID'])

pandas.core.series.Series

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Embedding, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, roc_auc_score
import ast
import numpy as np

# 将Input_ID从字符串转换为列表
m4_train['Input_ID'] = m4_train['Input_ID'].apply(ast.literal_eval)
m4_val['Input_ID'] = m4_val['Input_ID'].apply(ast.literal_eval)
m4_test['Input_ID'] = m4_test['Input_ID'].apply(ast.literal_eval)
# 提取数据
X_train = m4_train['Input_ID'].tolist()
y_train = m4_train['Label'].values

X_val = m4_val['Input_ID'].tolist()
y_val = m4_val['Label'].values

X_test = m4_test['Input_ID'].tolist()
y_test = m4_test['Label'].values

# 填充序列，确保所有序列的长度一致
maxlen = max(max([len(seq) for seq in X_train]), max([len(seq) for seq in X_val]), max([len(seq) for seq in X_test]))
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_val = pad_sequences(X_val, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

# 模型定义
model = Sequential([
    Embedding(input_dim=50000, output_dim=128, input_length=maxlen), # 假设字典大小为50000
    LSTM(64, return_sequences=True),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 模型训练
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# 预测
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# 计算accuracy和AUROC
accuracy = accuracy_score(y_test, y_pred)
auroc = roc_auc_score(y_test, y_pred_prob)

print(f'Accuracy: {accuracy}')
print(f'AUROC: {auroc}')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
2502/2502 ━━━━━━━━━━━━━━━━━━━━ 239s 92ms/step - accuracy: 0.9160 - loss: 0.2698 - val_accuracy: 0.8989 - val_loss: 0.2718
Epoch 2/10
2502/2502 ━━━━━━━━━━━━━━━━━━━━ 246s 87ms/step - accuracy: 0.9178 - loss: 0.2178 - val_accuracy: 0.9054 - val_loss: 0.2744
Epoch 3/10
2502/2502 ━━━━━━━━━━━━━━━━━━━━ 263s 88ms/step - accuracy: 0.9239 - loss: 0.1942 - val_accuracy: 0.9028 - val_loss: 0.2909
Epoch 4/10
2502/2502 ━━━━━━━━━━━━━━━━━━━━ 262s 88ms/step - accuracy: 0.9312 - loss: 0.1732 - val_accuracy: 0.8991 - val_loss: 0.3166
Epoch 5/10
2502/2502 ━━━━━━━━━━━━━━━━━━━━ 261s 87ms/step - accuracy: 0.9382 - loss: 0.1540 - val_accuracy: 0.8959 - val_loss: 0.3243
Epoch 6/10
2502/2502 ━━━━━━━━━━━━━━━━━━━━ 262s 87ms/step - accuracy: 0.9489 - loss: 0.1272 - val_accuracy: 0.8949 - val_loss: 0.3764
Epoch 7/10
2502/2502 ━━━━━━━━━━━━━━━━━━━━ 220s 88ms/step - accuracy: 0.9598 - loss: 0.1049 - val_accuracy: 0.8838 - val_loss: 0.4446
Epoch 8/10
2502/2502 ━━━━━━━━━━━━━━━━━━━━ 272s 92ms/step - accuracy: 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 306, 128)          24723712  
                                                                 
 lstm (LSTM)                 (None, 306, 128)          131584    
                                                                 
 dense (Dense)               (None, 306, 2)            258       
                                                                 
Total params: 24855554 (94.82 MB)
Trainable params: 24855554 (94.82 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# @title Default title text
PRETRAINED_MODEL_ARCHIVE_MAP = {
    'bert-base-uncased': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz",
    'bert-large-uncased': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased.tar.gz",
    'bert-base-cased': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased.tar.gz",
    'bert-base-multilingual': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual.tar.gz",
    'bert-base-chinese': "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese.tar.gz",
}
CONFIG_NAME = 'bert_config.json'
WEIGHTS_NAME = 'pytorch_model.bin'

def gelu(x):
    """Implementation of the gelu activation function.
        For information: OpenAI GPT's gelu is slightly different (and gives slightly different results):
        0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))
    """
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))


def swish(x):
    return x * torch.sigmoid(x)


ACT2FN = {"gelu": gelu, "relu": torch.nn.functional.relu, "swish": swish}
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

class BertLayerNorm(nn.Module):
    def __init__(self, config, variance_epsilon=1e-12):
        """Construct a layernorm module in the TF style (epsilon inside the square root).
        """
        super(BertLayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(config.hidden_size))
        self.beta = nn.Parameter(torch.zeros(config.hidden_size))
        self.variance_epsilon = variance_epsilon

    def forward(self, x):
        u = x.mean(-1, keepdim=True)
        s = (x - u).pow(2).mean(-1, keepdim=True)
        x = (x - u) / torch.sqrt(s + self.variance_epsilon)
        return self.gamma * x + self.beta


class BertEmbeddings(nn.Module):
    """Construct the embeddings from word, position and token_type embeddings.
    """
    def __init__(self, config):
        super(BertEmbeddings, self).__init__()
        self.word_embeddings = nn.Embedding(config.vocab_size, config.hidden_size)
        self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)
        self.token_type_embeddings = nn.Embedding(config.type_vocab_size, config.hidden_size)

        # self.LayerNorm is not snake-cased to stick with TensorFlow model variable name and be able to load
        # any TensorFlow checkpoint file
        self.LayerNorm = BertLayerNorm(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, input_ids, token_type_ids=None):
        seq_length = input_ids.size(1)
        position_ids = torch.arange(seq_length, dtype=torch.long, device=input_ids.device)
        position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
        if token_type_ids is None:
            token_type_ids = torch.zeros_like(input_ids)

        words_embeddings = self.word_embeddings(input_ids)
        position_embeddings = self.position_embeddings(position_ids)
        token_type_embeddings = self.token_type_embeddings(token_type_ids)

        embeddings = words_embeddings + position_embeddings + token_type_embeddings
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings


class BertSelfAttention(nn.Module):
    def __init__(self, config):
        super(BertSelfAttention, self).__init__()
        if config.hidden_size % config.num_attention_heads != 0:
            raise ValueError(
                "The hidden size (%d) is not a multiple of the number of attention "
                "heads (%d)" % (config.hidden_size, config.num_attention_heads))
        self.num_attention_heads = config.num_attention_heads
        self.attention_head_size = int(config.hidden_size / config.num_attention_heads)
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        self.query = nn.Linear(config.hidden_size, self.all_head_size)
        self.key = nn.Linear(config.hidden_size, self.all_head_size)
        self.value = nn.Linear(config.hidden_size, self.all_head_size)

        self.dropout = nn.Dropout(config.attention_probs_dropout_prob)

    def transpose_for_scores(self, x):
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, self.attention_head_size)
        x = x.view(*new_x_shape)
        return x.permute(0, 2, 1, 3)

    def forward(self, hidden_states, attention_mask):
        mixed_query_layer = self.query(hidden_states)
        mixed_key_layer = self.key(hidden_states)
        mixed_value_layer = self.value(hidden_states)

        query_layer = self.transpose_for_scores(mixed_query_layer)
        key_layer = self.transpose_for_scores(mixed_key_layer)
        value_layer = self.transpose_for_scores(mixed_value_layer)

        # Take the dot product between "query" and "key" to get the raw attention scores.
        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))
        attention_scores = attention_scores / math.sqrt(self.attention_head_size)
        # Apply the attention mask is (precomputed for all layers in BertModel forward() function)
        attention_scores = attention_scores + attention_mask

        # Normalize the attention scores to probabilities.
        attention_probs = nn.Softmax(dim=-1)(attention_scores)

        # This is actually dropping out entire tokens to attend to, which might
        # seem a bit unusual, but is taken from the original Transformer paper.
        attention_probs = self.dropout(attention_probs)

        context_layer = torch.matmul(attention_probs, value_layer)
        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        context_layer = context_layer.view(*new_context_layer_shape)
        return context_layer


class BertSelfOutput(nn.Module):
    def __init__(self, config):
        super(BertSelfOutput, self).__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.LayerNorm = BertLayerNorm(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, hidden_states, input_tensor):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        return hidden_states
class BertAttention(nn.Module):
    def __init__(self, config):
        super(BertAttention, self).__init__()
        self.self = BertSelfAttention(config)
        self.output = BertSelfOutput(config)

    def forward(self, input_tensor, attention_mask):
        self_output = self.self(input_tensor, attention_mask)
        attention_output = self.output(self_output, input_tensor)
        return attention_output


class BertIntermediate(nn.Module):
    def __init__(self, config):
        super(BertIntermediate, self).__init__()
        self.dense = nn.Linear(config.hidden_size, config.intermediate_size)
        self.intermediate_act_fn = ACT2FN[config.hidden_act] \
            if isinstance(config.hidden_act, str) else config.hidden_act

    def forward(self, hidden_states):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.intermediate_act_fn(hidden_states)
        return hidden_states


class BertOutput(nn.Module):
    def __init__(self, config):
        super(BertOutput, self).__init__()
        self.dense = nn.Linear(config.intermediate_size, config.hidden_size)
        self.LayerNorm = BertLayerNorm(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, hidden_states, input_tensor):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        return hidden_states
class BertLayer(nn.Module):
    def __init__(self, config):
        super(BertLayer, self).__init__()
        self.attention = BertAttention(config)
        self.intermediate = BertIntermediate(config)
        self.output = BertOutput(config)

    def forward(self, hidden_states, attention_mask):
        attention_output = self.attention(hidden_states, attention_mask)
        intermediate_output = self.intermediate(attention_output)
        layer_output = self.output(intermediate_output, attention_output)
        return layer_output

class BertEncoder(nn.Module):
    def __init__(self, config):
        super(BertEncoder, self).__init__()
        layer = BertLayer(config)
        self.layer = nn.ModuleList([copy.deepcopy(layer) for _ in range(config.num_hidden_layers)])

    def forward(self, hidden_states, attention_mask, output_all_encoded_layers=True):
        all_encoder_layers = []
        for layer_module in self.layer:
            hidden_states = layer_module(hidden_states, attention_mask)
            if output_all_encoded_layers:
                all_encoder_layers.append(hidden_states)
        if not output_all_encoded_layers:
            all_encoder_layers.append(hidden_states)
        return all_encoder_layers


class BertPooler(nn.Module):
    def __init__(self, config):
        super(BertPooler, self).__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()

    def forward(self, hidden_states):
        # We "pool" the model by simply taking the hidden state corresponding
        # to the first token.
        first_token_tensor = hidden_states[:, 0]
        pooled_output = self.dense(first_token_tensor)
        pooled_output = self.activation(pooled_output)
        return pooled_output

class PreTrainedBertModel(nn.Module):
    """ An abstract class to handle weights initialization and
        a simple interface for dowloading and loading pretrained models.
    """
    def __init__(self, config, *inputs, **kwargs):
        super(PreTrainedBertModel, self).__init__()
        if not isinstance(config, BertConfig):
            raise ValueError(
                "Parameter config in `{}(config)` should be an instance of class `BertConfig`. "
                "To create a model from a Google pretrained model use "
                "`model = {}.from_pretrained(PRETRAINED_MODEL_NAME)`".format(
                    self.__class__.__name__, self.__class__.__name__
                ))
        self.config = config

    def init_bert_weights(self, module):
        """ Initialize the weights.
        """
        if isinstance(module, (nn.Linear, nn.Embedding)):
            # Slightly different from the TF version which uses truncated_normal for initialization
            # cf https://github.com/pytorch/pytorch/pull/5617
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
        elif isinstance(module, BertLayerNorm):
            module.beta.data.normal_(mean=0.0, std=self.config.initializer_range)
            module.gamma.data.normal_(mean=0.0, std=self.config.initializer_range)
        if isinstance(module, nn.Linear) and module.bias is not None:
            module.bias.data.zero_()

    @classmethod
    def from_pretrained(cls, pretrained_model_name, *inputs, **kwargs):
        """
        Instantiate a PreTrainedBertModel from a pre-trained model file.
        Download and cache the pre-trained model file if needed.

        Params:
            pretrained_model_name: either:
                - a str with the name of a pre-trained model to load selected in the list of:
                    . `bert-base-uncased`
                    . `bert-large-uncased`
                    . `bert-base-cased`
                    . `bert-base-multilingual`
                    . `bert-base-chinese`
                - a path or url to a pretrained model archive containing:
                    . `bert_config.json` a configuration file for the model
                    . `pytorch_model.bin` a PyTorch dump of a BertForPreTraining instance
            *inputs, **kwargs: additional input for the specific Bert class
                (ex: num_labels for BertForSequenceClassification)
        """
        if pretrained_model_name in PRETRAINED_MODEL_ARCHIVE_MAP:
            archive_file = PRETRAINED_MODEL_ARCHIVE_MAP[pretrained_model_name]
        else:
            archive_file = pretrained_model_name
        # redirect to the cache, if necessary
            return None
        #if resolved_archive_file == archive_file:
            #logger.info("loading archive file {}".format(archive_file))
        #else:
           # logger.info("loading archive file {} from cache at {}".format(
             #   archive_file, resolved_archive_file))
        tempdir = None
        #if os.path.isdir(resolved_archive_file):
         #   serialization_dir = resolved_archive_file
        #else:
            # Extract archive to temp dir
          #  tempdir = tempfile.mkdtemp()
          #  logger.info("extracting archive file {} to temp dir {}".format(
          #      resolved_archive_file, tempdir))
          #  with tarfile.open(resolved_archive_file, 'r:gz') as archive:
          #      archive.extractall(tempdir)
          #  serialization_dir = tempdir
        # Load config
        #config_file = os.path.join(serialization_dir, CONFIG_NAME)
        #config = BertConfig.from_json_file(config_file)
        logger.info("Model config {}".format(config))
        # Instantiate model.
        model = cls(config, *inputs, **kwargs)
        #weights_path = os.path.join(serialization_dir, WEIGHTS_NAME)
        #state_dict = torch.load(weights_path, map_location = 'cpu')

        missing_keys = []
        unexpected_keys = []
        error_msgs = []
        # copy state_dict so _load_from_state_dict can modify it
        metadata = getattr(state_dict, '_metadata', None)
        state_dict = state_dict.copy()
        if metadata is not None:
            state_dict._metadata = metadata

class BertModel(PreTrainedBertModel):
    """BERT model ("Bidirectional Embedding Representations from a Transformer").

    Params:
        config: a BertConfig class instance with the configuration to build a new model

    Inputs:
        `input_ids`: a torch.LongTensor of shape [batch_size, sequence_length]
            with the word token indices in the vocabulary(see the tokens preprocessing logic in the scripts
            `extract_features.py`, `run_classifier.py` and `run_squad.py`)
        `token_type_ids`: an optional torch.LongTensor of shape [batch_size, sequence_length] with the token
            types indices selected in [0, 1]. Type 0 corresponds to a `sentence A` and type 1 corresponds to
            a `sentence B` token (see BERT paper for more details).
        `attention_mask`: an optional torch.LongTensor of shape [batch_size, sequence_length] with indices
            selected in [0, 1]. It's a mask to be used if the input sequence length is smaller than the max
            input sequence length in the current batch. It's the mask that we typically use for attention when
            a batch has varying length sentences.
        `output_all_encoded_layers`: boolean which controls the content of the `encoded_layers` output as described below. Default: `True`.

    Outputs: Tuple of (encoded_layers, pooled_output)
        `encoded_layers`: controled by `output_all_encoded_layers` argument:
            - `output_all_encoded_layers=True`: outputs a list of the full sequences of encoded-hidden-states at the end
                of each attention block (i.e. 12 full sequences for BERT-base, 24 for BERT-large), each
                encoded-hidden-state is a torch.FloatTensor of size [batch_size, sequence_length, hidden_size],
            - `output_all_encoded_layers=False`: outputs only the full sequence of hidden-states corresponding
                to the last attention block,
        `pooled_output`: a torch.FloatTensor of size [batch_size, hidden_size] which is the output of a
            classifier pretrained on top of the hidden state associated to the first character of the
            input (`CLF`) to train on the Next-Sentence task (see BERT's paper).

    Example usage:
    ```python
    # Already been converted into WordPiece token ids
    input_ids = torch.LongTensor([[31, 51, 99], [15, 5, 0]])
    input_mask = torch.LongTensor([[1, 1, 1], [1, 1, 0]])
    token_type_ids = torch.LongTensor([[0, 0, 1], [0, 2, 0]])

    config = modeling.BertConfig(vocab_size=32000, hidden_size=512,
        num_hidden_layers=8, num_attention_heads=6, intermediate_size=1024)

    model = modeling.BertModel(config=config)
    all_encoder_layers, pooled_output = model(input_ids, token_type_ids, input_mask)
    ```
    """
    def __init__(self, config):
        super(BertModel, self).__init__(config)
        self.embeddings = BertEmbeddings(config)
        self.encoder = BertEncoder(config)
        self.pooler = BertPooler(config)
        self.apply(self.init_bert_weights)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, output_all_encoded_layers=True):
        if attention_mask is None:
            attention_mask = torch.ones_like(input_ids)
        if token_type_ids is None:
            token_type_ids = torch.zeros_like(input_ids)

        # We create a 3D attention mask from a 2D tensor mask.
        # Sizes are [batch_size, 1, 1, to_seq_length]
        # So we can broadcast to [batch_size, num_heads, from_seq_length, to_seq_length]
        # this attention mask is more simple than the triangular masking of causal attention
        # used in OpenAI GPT, we just need to prepare the broadcast dimension here.
        extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)

        # Since attention_mask is 1.0 for positions we want to attend and 0.0 for
        # masked positions, this operation will create a tensor which is 0.0 for
        # positions we want to attend and -10000.0 for masked positions.
        # Since we are adding it to the raw scores before the softmax, this is
        # effectively the same as removing these entirely.
        extended_attention_mask = extended_attention_mask.to(dtype=next(self.parameters()).dtype) # fp16 compatibility
        extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0

        embedding_output = self.embeddings(input_ids, token_type_ids)
        encoded_layers = self.encoder(embedding_output,
                                      extended_attention_mask,
                                      output_all_encoded_layers=output_all_encoded_layers)
        sequence_output = encoded_layers[-1]
        pooled_output = self.pooler(sequence_output)
        if not output_all_encoded_layers:
            encoded_layers = encoded_layers[-1]
        return encoded_layers, pooled_output

Epoch 1/10
539/539 [==============================] - 40s 69ms/step - loss: 0.0221 - accuracy: 0.9959 - val_loss: 0.0107 - val_accuracy: 0.9973
Epoch 2/10
539/539 [==============================] - 18s 33ms/step - loss: 0.0040 - accuracy: 0.9995 - val_loss: 0.0205 - val_accuracy: 0.9970
Epoch 3/10
539/539 [==============================] - 15s 27ms/step - loss: 4.7524e-04 - accuracy: 1.0000 - val_loss: 0.0234 - val_accuracy: 0.9971
Epoch 4/10
539/539 [==============================] - 14s 26ms/step - loss: 1.5780e-04 - accuracy: 1.0000 - val_loss: 0.0254 - val_accuracy: 0.9970
Epoch 5/10
539/539 [==============================] - 13s 25ms/step - loss: 7.7871e-05 - accuracy: 1.0000 - val_loss: 0.0276 - val_accuracy: 0.9969
Epoch 6/10
539/539 [==============================] - 13s 24ms/step - loss: 4.5912e-05 - accuracy: 1.0000 - val_loss: 0.0294 - val_accuracy: 0.9967
Epoch 7/10
539/539 [==============================] - 13s 23ms/step - loss: 2.9397e-05 - accuracy: 1.0000 - val_loss: 0.

In [ ]:
# Load validation and test sets
val_file_path = "/content/drive/My Drive/m4_val_chunk.csv"
test_file_path = "/content/drive/My Drive/m4_test_chunk.csv"



674/674 [==============================] - 6s 9ms/step - loss: 0.0063 - accuracy: 0.9993
Validation Accuracy: 0.9992876052856445


674/674 [==============================] - 4s 6ms/step
